In [91]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
import cufflinks as cf
# print(cf.__version__)

In [92]:
reg = pd.read_csv('dfsent02.csv',encoding="ISO-8859-1")
reg.head(3)

,Unnamed: 0,title,author,views,shares,body,comments,alltags,sent_scaled,p,n,date,popularity
0,391,'Bitcoin is a currency': Federal judge says vi...,Cecilia Grigolia,94,0,"As it is known, Bitcoin is a digital currency,...",0,#Bitcoin News #Bitcoin Regulation #USA,-0.329151,2,10,2013-08-08,94
1,390,NY regulator may break down on Bitcoin,Cecilia Grigolia,106,0,New York financial regulators are concerned ab...,0,#Bitcoin Regulation #New York #USA ...,-0.216216,6,13,2013-08-12,106
2,798,Senate pressing regulators on Bitcoin risks an...,Eric Barrier,45,0,A Senate homeland security committee seeks exp...,0,#Bitcoin News #Bitcoin Regulation #USA,-0.104247,6,9,2013-08-13,45


In [93]:
prices = pd.read_csv('C:/Users/bridge/deda/regrisk/NLPnML/combined_df.csv')
prices.head(3)

,date,ETH,LTC,XRP,ETC,DASH,XMR,BCH,BTC
0,2015-01-01,NaN,2.711091,0.024008,NaN,1.978678,0.449231,NaN,316.768392
1,2015-01-02,NaN,2.685418,0.024511,NaN,1.974292,0.455994,NaN,315.785146
2,2015-01-03,NaN,2.389670,0.022297,NaN,1.818238,0.473959,NaN,301.512433


In [94]:
y15 = '2015-01-01'
y16 = '2016-01-01'
y17 = '2017-01-01'
y18 = '2018-01-01'
end = '2018-05-31'

In [95]:
def country_filter(country):
    number_find = []
    tag = str(country)
    for i in range(0,len(reg['alltags'])):
        if tag in str(reg['alltags'][i]):
            number_find.append(i)
    return number_find

# country_filter('#Japan')
def country_corr(country,i,fromdate,todate):
    reg_japan=reg.loc[reg.index.isin(country_filter(country))]
    reg_japan=reg_japan[['title','sent_scaled','date','popularity']]

    reg_japan.index = reg_japan['date']
    del(reg_japan['date'])
    prices.index = prices['date']
    pricesx = prices.fillna(0)
    pricesx=pricesx.drop(pricesx.columns[[0]], axis=1)

    prices1 = -pricesx.diff(periods=-1)
    prices7 = -pricesx.diff(periods=-i)
    prices30 = -pricesx.diff(periods=-30)

    cor_japan7 = reg_japan.merge(prices7, how='outer', left_index=True, right_index=True)
    cor_japan7 = cor_japan7[np.isfinite(cor_japan7['popularity'])]
    # cor_japan7.index = cor_japan7['date']
    # del(cor_japan7['date'])
    cor_japan7 = cor_japan7.drop(cor_japan7.columns[[0]], axis=1)
    cor_japan7_year = cor_japan7.loc[fromdate:todate]
    corr_country = cor_japan7_year
    return corr_country.pct_change().corr(method='pearson')

In [96]:
def correlation_heatmap(df, title, absolute_bounds=True):
    '''Plot a correlation heatmap for the entire dataframe'''
    heatmap = go.Heatmap(
        z=df.corr(method='pearson').as_matrix(),
        x=df.columns,
        y=df.columns,
        colorbar=dict(title='Pearson Coefficient'),
    )
    
    layout = go.Layout(title=title)
    
    if absolute_bounds:
        heatmap['zmax'] = 1.0
        heatmap['zmin'] = -1.0
        
    fig = go.Figure(data=[heatmap], layout=layout)
    py.iplot(fig)

In [106]:
def country_hm(country,i,fromdate,todate):
    reg_japan=reg.loc[reg.index.isin(country_filter(country))]
    reg_japan=reg_japan[['title','sent_scaled','date','popularity']]

    reg_japan.index = reg_japan['date']
    del(reg_japan['date'])
    prices.index = prices['date']
    pricesx = prices.fillna(0)
    pricesx=pricesx.drop(pricesx.columns[[0]], axis=1)

    prices1 = -pricesx.diff(periods=-1)
    prices7 = -pricesx.diff(periods=-i)
    prices30 = -pricesx.diff(periods=-30)

    cor_japan7 = reg_japan.merge(prices7, how='outer', left_index=True, right_index=True)
    cor_japan7 = cor_japan7[np.isfinite(cor_japan7['popularity'])]
    # cor_japan7.index = cor_japan7['date']
    # del(cor_japan7['date'])
    cor_japan7 = cor_japan7.drop(cor_japan7.columns[[0]], axis=1)
    cor_japan7_year = cor_japan7.loc[fromdate:todate]
    corr_country = cor_japan7_year
#     zz = country_corr.round(2)
#     z = zz.values
#     fig = ff.create_annotated_heatmap(z)
#     py.iplot(fig, filename='annotated_heatmap')
    return correlation_heatmap(corr_country, "Cryptocurrency Correlations in 2016")

In [98]:
country_corr('USA',1,y16,y17)

,sent_scaled,popularity,ETH,LTC,XRP,ETC,DASH,XMR,BCH,BTC
sent_scaled,1.000000,0.296624,0.035321,-0.092083,-0.372398,0.095711,-0.389267,-0.395963,NaN,-0.212007
popularity,0.296624,1.000000,0.289081,-0.474225,-0.642687,0.529249,0.461367,0.177869,NaN,-0.413203
ETH,0.035321,0.289081,1.000000,-0.263720,-0.066034,0.984932,0.196393,0.184253,NaN,-0.168508
LTC,-0.092083,-0.474225,-0.263720,1.000000,0.533581,0.278106,-0.741605,-0.409556,NaN,0.972330
XRP,-0.372398,-0.642687,-0.066034,0.533581,1.000000,-0.403830,-0.411031,-0.069849,NaN,0.506866
ETC,0.095711,0.529249,0.984932,0.278106,-0.403830,1.000000,-0.097238,-0.603158,NaN,0.257497
DASH,-0.389267,0.461367,0.196393,-0.741605,-0.411031,-0.097238,1.000000,0.707100,NaN,-0.620604
XMR,-0.395963,0.177869,0.184253,-0.409556,-0.069849,-0.603158,0.707100,1.000000,NaN,-0.372398
BCH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BTC,-0.212007,-0.413203,-0.168508,0.972330,0.506866,0.257497,-0.620604,-0.372398,NaN,1.000000


In [121]:
country_hm('South Korea',1,y17,y18)

C:\Users\bridge\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

